In [1]:
import pyodbc
import pandas as pd
from sklearn.linear_model import LinearRegression


# Detalles de conexión
server = '.'
database = 'Testing'
#username = ''
#password = ''
driver= '{ODBC Driver 17 for SQL Server}' 
        
# Creación de la cadena de conexión
conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')


# consulta SQL
query = """

DECLARE @FechaMaxima DATETIME;
SELECT @FechaMaxima = MAX([Fecha])
FROM Ventas.dbo.[venta_detalle];

SELECT 
    [Fecha]
    ,[Sucursal]
    ,[Artículo]
	,SUBSTRING(ClaseS,2,1) as Frecuencia
FROM 
    [Testing].[dbo].[Clasificacion_test]
WHERE 
    Fecha = @FechaMaxima 
ORDER BY 
    Artículo, Sucursal, Fecha;

"""

# Ejecutar  consulta y cargar  datos en un DataFrame
df = pd.read_sql(query, conn)

C:\Users\diego.salinas\AppData\Local\Temp\ipykernel_36284\1844772744.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
# Mapeo de clasificaciones a valores numéricos
clasificacion_a_numero = {'A': 4, 'B': 2, 'C': 1, 'L': 0}
df['Frecuencia_Num'] = df['Frecuencia'].map(clasificacion_a_numero)

# Convertir las fechas a formato numérico para la regresión
df['Fecha_Num'] = pd.to_datetime(df['Fecha']).map(pd.Timestamp.toordinal)

# Agrupar datos por Artículo y Sucursal
grouped = df.groupby(['Artículo', 'Sucursal'])

# Función para aplicar regresión lineal a cada grupo
#def apply_linear_regression(group):
 #   if group['Frecuencia_Num'].nunique() > 1:  # Más de un valor único
  #      model = LinearRegression()
   #     X = group['Fecha_Num'].values.reshape(-1, 1)
    #    y = group['Frecuencia_Num'].values
     #   model.fit(X, y)
      #  return model.coef_[0]  # Retorna la pendiente
 #   else:
  #      return 0.0  # Retorna 0 si todos los valores son iguales

def apply_linear_regression(group):
    if group['Frecuencia_Num'].nunique() > 1:  # Más de un valor único
        model = LinearRegression()
        X = group['Fecha_Num'].values.reshape(-1, 1)
        y = group['Frecuencia_Num'].values
        model.fit(X, y)
        return pd.DataFrame({'Tendencia': [model.coef_[0]] * len(group)})  # Retorna la pendiente
    else:
        return pd.DataFrame({'Tendencia': [0.0] * len(group)})  # Retorna 0 si todos los valores son iguales


# Calcular las tendencias para cada grupo
trends = grouped.apply(apply_linear_regression).reset_index()

# Fusionar los resultados con el DataFrame original
df = pd.merge(df, trends, on=['Artículo', 'Sucursal'], how='left')

# Mostrar los primeros registros del DataFrame con la tendencia calculada
print(df.head())

ValueError: cannot insert Sucursal, already exists

In [ ]:
cursor = conn.cursor()

# Preparar y ejecutar las instrucciones de actualización
for index, row in df.iterrows():
    update_query = """
    UPDATE Clasificacion_test
    SET Tendencia = ?
    WHERE Fecha = ? AND Sucursal = ? AND Artículo = ?
    """
    cursor.execute(update_query, (row['Tendencia'], row['Fecha'], row['Sucursal'], row['Artículo']))

conn.commit()  # Confirmar todos los cambios
cursor.close()  # Cerrar el cursor
conn.close()  # Cerrar la conexión

print("La tabla ha sido actualizada con éxito.")